In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('Template1.xlsx',sheet_name = None)

In [3]:
xls = pd.ExcelFile('Template1.xlsx')
xls.sheet_names 
flow = data.get('Flow vs. Group')
flow.head()

,SN,Flow,Step Name,WS,Area,Step SEQ NO,STEP STATUS,REC TO SCHED?,EXPORT TO CRINV?
0,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU31_DHF100,F11 WET PROCESS,100,ACTIVE,Y,Y
1,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU3_DHF100,F11 WET PROCESS,100,ACTIVE,Y,N
2,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU31_DHF_4,F11 WET PROCESS,100,ACTIVE,Y,N
3,2,100S PC,5200-53 GATE NITRIDE DEP 2 CMP,11-3EBRA_FRXX_BUFF,F11 CMP,200,ACTIVE,Y,N
4,2,100S PC,5200-53 GATE NITRIDE DEP 2 CMP,11-3EBRA_FRXS_BUFF,F11 CMP,200,ACTIVE,Y,Y


In [4]:
process = input('Please Enter the product: ')

Please Enter the product: 110S PC


In [8]:
flow = flow[flow['Flow'] == process]
flow.head()

,SN,Flow,Step Name,WS,Area,Step SEQ NO,STEP STATUS,REC TO SCHED?,EXPORT TO CRINV?
1894,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU31_DHF100,F11 WET PROCESS,200,ACTIVE,Y,N
1895,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU3_DHF100,F11 WET PROCESS,200,ACTIVE,Y,N
1896,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU32_DHF_SPM,F11 WET PROCESS,200,ACTIVE,Y,N
1897,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU32_100HF,F11 WET PROCESS,200,ACTIVE,Y,N
1898,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU31_DHF_4,F11 WET PROCESS,200,ACTIVE,Y,Y


In [9]:
Qty = data.get('WS vs. Tool ID') 
WS_NAME = Qty['WS']
WS_NAME1 = []
for i in WS_NAME:
    WS_NAME1.append(i.replace('\xa0', ''))#去除字尾空格
print(len(WS_NAME1))

3780


In [10]:
G_WS = data.get('Group vs. WS')
G_WS1 = G_WS.set_index(['Capacity Group'],inplace = False)
WS_Group = []
for i in WS_NAME1:
    if i in G_WS1['WS'].tolist():
        group = G_WS1[G_WS1['WS'] ==  i].index.tolist()
    else:
        group = ['0']
    #group = G_WS[G_WS['WS'] ==  i].iat[0,0]
    WS_Group.extend(group)
    #print(group)
Qty['WS Group'] = WS_Group
Qty.head()

,MiCAP ID,WS,Tool ID,Location,WS Group
0,1195118,11-3HIKE_Q2LV_UDPLY,KISDB7VA00,7,11-DIF_UDPLY
1,1195119,11-3HIKE_Q2LV_BSIGE,KISDB7V300,7,11-DIF_BSIGEPLY
2,1195249,11-3TEL_SDRM_EBK,TTSDBC0130,C,11-DRY_SDRM_EBK
3,1195249,11-3TEL_SDRM_EBK,TTSDBC0120,C,11-DRY_SDRM_EBK
4,1195249,11-3TEL_SDRM_EBK,TTSDBC0110,C,11-DRY_SDRM_EBK


In [11]:
step = []
for i in flow['WS']:
    if i in G_WS1['WS'].tolist():
        s = G_WS1[G_WS1['WS'] ==  i].index.tolist()
    else:
        s = ['0']
    step.extend(s)
#print(step.values.tolist())
#print(step)

In [12]:
flow = flow.drop(['Area','Step SEQ NO','STEP STATUS','REC TO SCHED?','EXPORT TO CRINV?'], axis = 1)
flow['WS Group'] = step
flow.head()

,SN,Flow,Step Name,WS,WS Group
1894,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU31_DHF100,11-WET_SWC_DHF
1895,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU3_DHF100,11-WET_SWC_DHF
1896,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU32_DHF_SPM,11-WET_SWC_SPM_APM
1897,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU32_100HF,11-WET_SWC_SPM_APM
1898,1,110S PC,5430-49 SPACER OXIDE DRY STRIP RINSE,11-3DNS_SU31_DHF_4,11-WET_SWC_DHF


In [13]:
wafer = input('Please enter the quantity of wafers:')
wafer_m = []
for i in step:
    if i in WS_Group:
        w = round(int(wafer)/Qty.groupby(['WS Group']).size()[i],2)
        wafer_m.append(w)
    else:
        wafer_m.append(0)
    #print(i)
#print ('Number of wafers in each machine:',wafer_m)

Please enter the quantity of wafers:16000


In [14]:
add_WS = input('Please enter the Workstation Group to add tool : ') #增加機台的 Workstation
included_WS = [i for i,x in enumerate(step) if x==add_WS] #重複項索引值
print((included_WS))
#add_machine = input('Please enter the quantity of equipment added: ')
#change_data = wafer_in_each_machine.iloc[int(change_step)-1:int(change_step)+2,] #提取該步驟
change_WS = []
for i in included_WS:
    _WS_ = step[i-1:i+2] #提取索引
    change_WS.append(_WS_)
print(change_WS)

Please enter the Workstation Group to add tool : 11-DIF_THOX
[17]
[['11-WET_BEVEL_CF4_CW3', '11-DIF_THOX', '11-WET_SCRUB_FS']]


In [16]:
col = ['3','4','7','8','9','C','D','E']
#print(wafer_in_each_machine)
wafer_in_each_machine = pd.DataFrame(columns = col)
machine_num = pd.DataFrame(columns = col)
for i in change_WS:
    for j in range(3):
        f1 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '3')].shape[0]
        f2 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '4')].shape[0]
        f3 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '7')].shape[0]
        f4 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '8')].shape[0]
        f5 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '9')].shape[0]
        f6 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == 'C')].shape[0]
        f7 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == 'D')].shape[0]
        f8 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == 'E')].shape[0]
        machine = np.array([f1,f2,f3,f4,f5,f6,f7,f8])
        machine_D = pd.DataFrame([machine],columns = col)
        machine_num = machine_num.append(machine_D)
        print(machine_num)
        
        for k in range(8):
            if i[j] in WS_Group:
                num = wafer_m[step.index(i[j])]*machine[k]
            else:
                num = 0
            print(wafer_m[step.index(i[j])])
            machine[k] = num
        machine = pd.DataFrame([machine],columns = col)
        #print(machine)
        wafer_in_each_machine = wafer_in_each_machine.append(machine)
wafer_in_each_machine.head()
#print(wafer_in_each_machine.shape[0])  

   3  4  7  8  9  C  D  E
0  1  4  0  0  0  0  0  0
3200.0
3200.0
3200.0
3200.0
3200.0
3200.0
3200.0
3200.0
   3  4  7  8  9  C  D  E
0  1  4  0  0  0  0  0  0
0  0  0  2  0  0  5  0  0
2285.71
2285.71
2285.71
2285.71
2285.71
2285.71
2285.71
2285.71
    3  4   7  8  9  C  D  E
0   1  4   0  0  0  0  0  0
0   0  0   2  0  0  5  0  0
0  14  0  14  0  0  7  1  0
444.44
444.44
444.44
444.44
444.44
444.44
444.44
444.44


,3,4,7,8,9,C,D,E
0,3200,12800,0,0,0,0,0,0
0,0,0,4571,0,0,11428,0,0
0,6222,0,6222,0,0,3111,444,0


In [17]:
option1 = np.array([[1, 0, 4, 0, 0, 0, 0, 0],[0, 0, 1, 0, 0, 4, 0, 0],[14, 0, 14, 0, 0, 7, 1, 0]])
option2 = np.array([[1, 0, 4, 0, 0, 0, 0, 0],[0, 0, 2, 0, 0, 3, 0, 0],[14, 0, 14, 0, 0, 7, 1, 0]])
option = np.vstack((option1,option2))
#print(option)
num_each = [3200,3200,444.44]
for i in range(3):
    for j in range(8):
        option[i,j] = num_each[i]*option[i,j]
        option[i+3,j] = num_each[i]*option[i+3,j]
#print(option)
option = np.vstack((wafer_in_each_machine.values,option))   
print(option)
data = np.vsplit(option, 3)

[[3200 12800 0 0 0 0 0 0]
 [0 0 4571 0 0 11428 0 0]
 [6222 0 6222 0 0 3111 444 0]
 [3200 0 12800 0 0 0 0 0]
 [0 0 3200 0 0 12800 0 0]
 [6222 0 6222 0 0 3111 444 0]
 [3200 0 12800 0 0 0 0 0]
 [0 0 6400 0 0 9600 0 0]
 [6222 0 6222 0 0 3111 444 0]]


In [18]:
def weight(distance,carry):
    total_carry = []
    for i in range(2):
        carry_weight = np.multiply(carry[i],distance)
        total_carry_step = np.sum(carry_weight)-np.trace(carry_weight) 
        total_carry.append(total_carry_step)
    return sum(total_carry)
weight_distance = np.array([[0,1,1,2,3,1,2,3],[1,0,2,3,4,2,3,4],[1,2,0,1,2,1,2,3],[2,3,1,0,1,2,3,4]
                           ,[3,4,2,1,0,3,4,5],[1,2,1,2,3,0,1,2],[2,3,2,3,4,1,0,1],[3,4,3,4,5,2,1,0]])
Carriage = []
for l in range(3):
    a = range(len(col))
    c_m = []
    for k in range(2):
        for j in a:
            for i in a:
                c_n = max(data[l][k,j]-data[l][k+1,j],0)*data[l][k+1,a]//(int(wafer)-data[l][k+1,j]+1) #避免分母為零
            c_m.append(c_n)      
    c_step_m = np.array(c_m)
    #print(c_step_m)
    #print(c_step_m.shape)
    c_carry = np.vsplit(c_step_m, 2)
    for i in range(2):
        for j in range(8):
            c_carry[i][j][j] = min(data[l][i,j],data[l][i+1,j])
            c_carry = np.array(c_carry)
    Carriage.append(weight(distance = weight_distance,carry = c_carry))
Carriage = np.array([Carriage]).reshape(3,1)
Carriage = pd.DataFrame(Carriage,columns = ['Carriage'])
df = pd.DataFrame({'option': ['origin','keep Fab11C 3F','keep Fab11B 3F']})
Carriage = pd.concat([df,Carriage],axis = 1)
Carriage = Carriage.set_index('option')
Carriage.head(3)

,Carriage
option,
origin,37107
keep Fab11C 3F,22482
keep Fab11B 3F,16269
